In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from tqdm import tqdm
import shutil
from torchvision import datasets, transforms

from model import ResNet50, ResNet38, ResNet26



In [2]:

## 데이터 셋은 CIFAR10으로
num_classes = 10
batch_size = 10
num_workers = 4

## 데이터셋 로드
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.4914, 0.4822, 0.4465),
        std=(0.2470, 0.2435, 0.2616)
    )
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.4914, 0.4822, 0.4465),
        std=(0.2470, 0.2435, 0.2616)
    )
])
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('data', train=True, download=True, transform=transform_train),
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers
)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('data', train=False, transform=transform_test),
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers
)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data


In [3]:
stem = False
model = ResNet26(num_classes=num_classes, stem=stem)
model = model.cuda()

In [4]:
start_epoch = 1
best_acc = 0.0

lr = 0.1
momentum =0.9
weight_decay = 0.0001


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)


In [5]:
epochs = 100
print_interval = 20

## 30에폭 마다 러닝레이트 줄임
def adjust_learning_rate(optimizer, epoch):
    decayed_lr = lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = decayed_lr

for epoch in range(1,epochs+1):
    model.train()
    train_acc = 0.0
    step = 0
    for data, target in train_loader:
        adjust_learning_rate(optimizer, epoch)
        data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()


        ## onnx 저장


        y_pred = output.data.max(1)[1]
        
        acc = float(y_pred.eq(target.data).sum()) / len(data) * 100.
        train_acc += acc
        step += 1
        if step % print_interval == 0:
            print("Epoch ",epoch," ",len(train_loader)," / ",step,"batch Loss: ",loss.item(),"  ACC: ",acc,"%")

    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.cuda(), target.cuda()
            output = model(data)
            prediction = output.data.max(1)[1]
            correct += prediction.eq(target.data).sum()

    acc = 100. * float(correct) / len(test_loader.dataset)
    print('Test acc: {0:.2f} \n'.format(acc))

Epoch  1   5000  /  20 batch Loss:  9.364157676696777   ACC:  20.0 %
Epoch  1   5000  /  40 batch Loss:  3.7577877044677734   ACC:  0.0 %
Epoch  1   5000  /  60 batch Loss:  2.3191561698913574   ACC:  0.0 %
Epoch  1   5000  /  80 batch Loss:  2.3303494453430176   ACC:  10.0 %
Epoch  1   5000  /  100 batch Loss:  2.426391124725342   ACC:  10.0 %
Epoch  1   5000  /  120 batch Loss:  2.3636438846588135   ACC:  0.0 %
Epoch  1   5000  /  140 batch Loss:  2.285841941833496   ACC:  10.0 %
Epoch  1   5000  /  160 batch Loss:  2.2595152854919434   ACC:  20.0 %
Epoch  1   5000  /  180 batch Loss:  2.335702896118164   ACC:  10.0 %
Epoch  1   5000  /  200 batch Loss:  2.3778865337371826   ACC:  0.0 %
Epoch  1   5000  /  220 batch Loss:  2.3104355335235596   ACC:  10.0 %
Epoch  1   5000  /  240 batch Loss:  2.1990444660186768   ACC:  20.0 %
Epoch  1   5000  /  260 batch Loss:  2.32319974899292   ACC:  10.0 %
Epoch  1   5000  /  280 batch Loss:  2.249321699142456   ACC:  10.0 %
Epoch  1   5000  /  3

KeyboardInterrupt: 

In [ ]:
## onnx 저장
## operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK 꼭 해줘야함
## unfold 연산을 
# https://github.com/pytorch/pytorch/issues/14395#issuecomment-444697403


# import torch.onnx
# torch.onnx.export(model.cpu(),data.cpu(),'./stand_alone_self.onnx',export_params=False,opset_version=12,
#                   operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK)

In [ ]:
print(model.ch)